# Install packages
In this example, I use chainer chemistry which offer an implementation of SchNet.
This library can be install by PIP.
* Chainer Chemistry: A Library for Deep Learning in Biology and Chemistry<br>https://github.com/pfnet-research/chainer-chemistry

In [27]:
# !pip uninstall -y tensorflow
# !pip install chainer-chemistry==0.5.0

# Import packages
Next, I import main packages. Other sub-modules are imported later.

In [28]:
import random
import numpy as np
import pandas as pd

In [29]:
import chainer
import chainer_chemistry
from IPython.display import display

# Load dataset
In this example, 90% of training data is used actual training data, and the other 10% is used for validation.
Each dataset is grouped by molecule_name name for following procedures.

### Distances!

In [30]:
def build_type_dataframes(base, structures, coupling_type):
    base = base[base['type'] == coupling_type].drop('type', axis=1).copy()
    base = base.reset_index()
    base['id'] = base['id'].astype('int32')
    structures = structures[structures['molecule_index'].isin(base['molecule_index'])]
    return base, structures


In [31]:
def load_dataset():

    train = pd.merge(pd.read_csv('schnet_newdata/new_train.csv'),
                     pd.read_csv('../champs-scalar-coupling/scalar_coupling_contributions.csv'))

    test = pd.read_csv('schnet_newdata/new_test.csv')

    counts = train['molecule_name'].value_counts()
    moles = list(counts.index)

    random.shuffle(moles)

    num_train = int(len(moles) * 0.9)
    train_moles = sorted(moles[:num_train])
    valid_moles = sorted(moles[num_train:])
    test_moles = sorted(list(set(test['molecule_name'])))

    valid = train.query('molecule_name not in @train_moles')
    train = train.query('molecule_name in @train_moles')

    train.sort_values('molecule_name', inplace=True)
    valid.sort_values('molecule_name', inplace=True)
    test.sort_values('molecule_name', inplace=True)

    return train, valid, test, train_moles, valid_moles, test_moles

train, valid, test, train_moles, valid_moles, test_moles = load_dataset()

train_gp = train.groupby('molecule_name')
valid_gp = valid.groupby('molecule_name')
test_gp = test.groupby('molecule_name')

structures = pd.read_csv('../champs-scalar-coupling/structures.csv')
structures_groups = structures.groupby('molecule_name')

d:\prolificacy\Anaconda3\envs\pytorch\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [32]:
train_gp.groups

{'dsgdb9nsd_000001': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'dsgdb9nsd_000002': [10, 11, 12, 13, 14, 15], 'dsgdb9nsd_000003': [16], 'dsgdb9nsd_000005': [17, 18], 'dsgdb9nsd_000006': [19, 20], 'dsgdb9nsd_000007': [37, 38, 39, 40, 41, 43, 36, 44, 45, 46, 47, 42, 35, 26, 33, 21, 22, 23, 24, 34, 25, 28, 29, 30, 31, 32, 27], 'dsgdb9nsd_000008': [54, 57, 56, 55, 53, 52, 51, 50, 49, 48], 'dsgdb9nsd_000009': [72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58], 'dsgdb9nsd_000010': [84, 83, 82, 81, 80, 79, 77, 76, 75, 74, 73, 78], 'dsgdb9nsd_000013': [130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 143, 142, 144, 145, 146, 147, 148, 149, 150, 141, 129, 139, 127, 108, 109, 111, 112, 113, 114, 115, 116, 117, 110, 128, 118, 126, 124, 123, 125, 121, 120, 119, 122], 'dsgdb9nsd_000016': [170, 175, 171, 172, 173, 174, 176, 181, 178, 179, 180, 182, 183, 169, 177, 168, 159, 166, 151, 152, 153, 154, 155, 167, 157, 156, 160, 161, 162, 163, 164, 165, 158], 'dsgdb9nsd_000017': [192, 196, 195, 194, 197, 1

## train data

In [33]:
display(train.head(10))

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,83.0224,0.254579,1.25862,0.272010
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,-11.0347,0.352978,2.85839,-3.433600
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,-11.0325,0.352944,2.85852,-3.433870
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,-11.0319,0.352934,2.85855,-3.433930
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,83.0222,0.254585,1.25861,0.272013
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541,-11.0317,0.352932,2.85856,-3.433950
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548,-11.0324,0.352943,2.85853,-3.433870
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,83.0241,0.254634,1.25856,0.272012
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543,-11.0319,0.352943,2.85856,-3.433930
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,83.0243,0.254628,1.25856,0.272012


## validation data

In [34]:
display(valid.head(10))

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
85,95,dsgdb9nsd_000011,5,1,2JHC,-3.771720,-3.779600,0.162750,-0.037382,-0.117490
98,85,dsgdb9nsd_000011,3,0,1JHC,88.283700,86.657300,0.226042,0.821762,0.578633
96,87,dsgdb9nsd_000011,3,4,2JHH,-10.288000,-10.230600,0.329742,2.522750,-2.909880
95,88,dsgdb9nsd_000011,3,5,2JHH,-10.338300,-10.283700,0.330377,2.520420,-2.905390
94,89,dsgdb9nsd_000011,3,6,3JHH,6.792040,7.790410,-0.033392,2.038100,-3.003080
93,90,dsgdb9nsd_000011,4,0,1JHC,82.514100,80.670900,0.174409,1.099370,0.569410
92,91,dsgdb9nsd_000011,4,1,2JHC,-3.781050,-3.788060,0.162387,-0.038622,-0.116760
97,86,dsgdb9nsd_000011,3,1,2JHC,-6.566890,-6.167880,0.013447,-0.218375,-0.194087
90,96,dsgdb9nsd_000011,5,6,3JHH,0.188092,0.206961,0.083462,0.622025,-0.724356
89,93,dsgdb9nsd_000011,4,6,3JHH,0.198425,0.215153,0.084005,0.618686,-0.719418


## test data

In [35]:
display(test.head(10))

,id,molecule_name,atom_index_0,atom_index_1,type
0,198,dsgdb9nsd_000018,4,0,1JHC
23,203,dsgdb9nsd_000018,5,0,1JHC
22,207,dsgdb9nsd_000018,6,0,1JHC
21,206,dsgdb9nsd_000018,5,6,2JHH
20,205,dsgdb9nsd_000018,5,2,3JHC
18,201,dsgdb9nsd_000018,4,5,2JHH
17,202,dsgdb9nsd_000018,4,6,2JHH
16,200,dsgdb9nsd_000018,4,2,3JHC
15,199,dsgdb9nsd_000018,4,1,2JHC
14,209,dsgdb9nsd_000018,6,2,3JHC


## structures

In [36]:
display(structures.head(10))

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602


In [37]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)
valid.to_csv('valid.csv', index=False)
structures.to_csv('structures.csv', index=False)
!ls 

'ls' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


# Preprocessing
I implemented a class named `Graph` whose instances contain molecules.
The distances between atoms are calculated in the initializer of this class.
## Define Graph class

In [38]:
from scipy.spatial import distance


class Graph:

    def __init__(self, points_df, list_atoms, molecule):

        self.name = molecule
        
        self.points = points_df[['x', 'y', 'z']].values

        self._dists = distance.cdist(self.points, self.points)

        self.adj = self._dists < 1.5
        self.num_nodes = len(points_df)
        
        self.natoms = self.num_nodes
        
        self.atoms = points_df['atom']
        dict_atoms = {at: i for i, at in enumerate(list_atoms)}

        atom_index = [dict_atoms[atom] for atom in self.atoms]
        one_hot = np.identity(len(dict_atoms))[atom_index]

        bond = np.sum(self.adj, 1) - 1
        bonds = np.identity(len(dict_atoms))[bond - 1]

        self._array = np.concatenate([one_hot, bonds], axis=1).astype(np.float32)

    @property
    def input_array(self):
        return self._array

    @property
    def dists(self):
        return self._dists.astype(np.float32)

## Convert into graph object
Each dataset is represented as a list of Graphs and prediction targets.

In [1]:
list_atoms = list(set(structures['atom']))
print (list_atoms)
for mole in train_moles:
    a = Graph(structures_groups.get_group(mole), list_atoms, mole)
    break
    
print(a.name)
print(a.atoms)

NameError: name 'structures' is not defined

In [40]:
a.input_array

array([[0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.]], dtype=float32)

In [41]:
a.dists

array([[0.       , 1.091953 , 1.0919516, 1.0919464, 1.0919476],
       [1.091953 , 0.       , 1.7831198, 1.7831475, 1.7831566],
       [1.0919516, 1.7831198, 0.       , 1.7831577, 1.7831484],
       [1.0919464, 1.7831475, 1.7831577, 0.       , 1.7831479],
       [1.0919476, 1.7831566, 1.7831484, 1.7831479, 0.       ]],
      dtype=float32)

In [42]:
%%time

list_atoms = list(set(structures['atom']))
print('list of atoms')
print(list_atoms)
    
train_graphs = list()
train_targets = list()
print('preprocess training molecules ...')
for mole in train_moles:
    train_graphs.append(Graph(structures_groups.get_group(mole), list_atoms, mole))
    train_targets.append(train_gp.get_group(mole))

valid_graphs = list()
valid_targets = list()
print('preprocess validation molecules ...')
for mole in valid_moles:
    valid_graphs.append(Graph(structures_groups.get_group(mole), list_atoms, mole))
    valid_targets.append(valid_gp.get_group(mole))

test_graphs = list()
test_targets = list()
print('preprocess test molecules ...')
for mole in test_moles:
    test_graphs.append(Graph(structures_groups.get_group(mole), list_atoms, mole))
    test_targets.append(test_gp.get_group(mole))

list of atoms
['F', 'N', 'O', 'C', 'H']
preprocess training molecules ...
preprocess validation molecules ...
preprocess test molecules ...
Wall time: 1min 7s


In [43]:
print (len(train_graphs), len(train_targets))

68859 68859


In [44]:
print (train_graphs[0])
train_graphs[0].atoms

0    C
1    H
2    H
3    H
4    H
Name: atom, dtype: object

In [45]:
train_graphs[0].input_array

array([[0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.]], dtype=float32)

In [46]:
train_graphs[0].dists

array([[0.       , 1.091953 , 1.0919516, 1.0919464, 1.0919476],
       [1.091953 , 0.       , 1.7831198, 1.7831475, 1.7831566],
       [1.0919516, 1.7831198, 0.       , 1.7831577, 1.7831484],
       [1.0919464, 1.7831475, 1.7831577, 0.       , 1.7831479],
       [1.0919476, 1.7831566, 1.7831484, 1.7831479, 0.       ]],
      dtype=float32)

### Targets!

In [47]:
train_targets[1]

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,fc,sd,pso,dso
10,10,dsgdb9nsd_000002,1,0,1JHN,32.6889,30.6116,0.059952,1.94935,0.067923
11,11,dsgdb9nsd_000002,1,2,2JHH,-11.1866,-11.8280,0.574885,5.01783,-4.951410
12,12,dsgdb9nsd_000002,1,3,2JHH,-11.1757,-11.8157,0.574494,5.01889,-4.953350
13,13,dsgdb9nsd_000002,2,0,1JHN,32.6891,30.6119,0.059961,1.94935,0.067922
14,14,dsgdb9nsd_000002,2,3,2JHH,-11.1758,-11.8159,0.574501,5.01890,-4.953350
15,15,dsgdb9nsd_000002,3,0,1JHN,32.6905,30.6135,0.059939,1.94911,0.067931


## Convert into chainer's dataset
This type of dataset can be handled by `DictDataset`.
Graph objects and prediction targets are merged as a `DictDataset`.

In [48]:
from chainer.datasets.dict_dataset import DictDataset

train_dataset = DictDataset(graphs=train_graphs, targets=train_targets)
valid_dataset = DictDataset(graphs=valid_graphs, targets=valid_targets)
test_dataset = DictDataset(graphs=test_graphs, targets=test_targets)

In [49]:
train_dataset

# Model
## Build SchNet model
The prediction model is implemented as follows.
First, fully connected layer is applied to input arrays to align dimensions.
Next, SchNet layer is applied for feature extraction.
Finally, features vectors are concatenated and thrown into three layers MLP.
I add batch-normalization layers like ResNet.

In [50]:
from chainer import reporter
from chainer import functions as F
from chainer import links as L
from chainer_chemistry.links import SchNetUpdate
from chainer_chemistry.links import GraphLinear, GraphBatchNormalization

In [51]:


class SchNetUpdateBN(SchNetUpdate):

    def __init__(self, *args, **kwargs):
        super(SchNetUpdateBN, self).__init__(*args, **kwargs)
        with self.init_scope():
            self.bn = GraphBatchNormalization(args[0])

    def __call__(self, h, adj, **kwargs):
        v = self.linear[0](h)
        v = self.cfconv(v, adj)
        v = self.bn(v)
        v = self.linear[1](v)
        v = F.softplus(v)
        v = self.bn(v)
        v = self.linear[2](v)
        return h + self.bn(v)

class SchNet(chainer.Chain):

    def __init__(self, num_layer=3):
        super(SchNet, self).__init__()

        self.num_layer = num_layer

        with self.init_scope():
            self.gn = GraphLinear(512)
            for l in range(self.num_layer):
                self.add_link('sch{}'.format(l), SchNetUpdateBN(512))

            self.interaction1 = L.Linear(512)
            self.interaction2 = L.Linear(256)
            self.interaction3 = L.Linear(4)

    def __call__(self, input_array, dists, pairs_index, targets):

        out = self.predict(input_array, dists, pairs_index)
        loss = F.mean_absolute_error(out, targets)
        reporter.report({'loss': loss}, self)
        return loss

    def predict(self, input_array, dists, pairs_index, **kwargs):

        h = self.gn(input_array)

        for l in range(self.num_layer):
            h = self['sch{}'.format(l)](h, dists)

        h = F.concat((h, input_array), axis=2)

        concat = F.concat([
            h[pairs_index[:, 0], pairs_index[:, 1], :],
            h[pairs_index[:, 0], pairs_index[:, 2], :],
            F.expand_dims(dists[pairs_index[:, 0],
                                pairs_index[:, 1],
                                pairs_index[:, 2]], 1)
        ], axis=1)

        h1 = F.relu(self.interaction1(concat))
        h2 = F.relu(self.interaction2(h1))
        out = self.interaction3(h2)

        return out

model = SchNet(num_layer=3)
model.to_gpu(device=0)

# Training preparation
## Make samplers
For mini-batch training, I implement a sampler named `SameSizeSampler`.
The molecules which have same number of atoms are selected simultaneously.

In [52]:
np.random.random.__self__

RandomState(MT19937) at 0x1FCA01EA8C8

In [53]:
from chainer.iterators import OrderSampler

class SameSizeSampler(OrderSampler):

    def __init__(self, structures_groups, moles, batch_size,
                 random_state=42, use_remainder=False):

        self.structures_groups = structures_groups
        self.moles = moles
        self.batch_size = batch_size
        if random_state is None:
            random_state = np.random.random.__self__
        self._random = random_state
        self.use_remainder = use_remainder

    def __call__(self, current_order, current_position):

        batches = list()

        atom_counts = pd.DataFrame()
        atom_counts['mol_index'] = np.arange(len(self.moles))
        atom_counts['molecular_name'] = self.moles
        atom_counts['num_atom'] = [len(self.structures_groups.get_group(mol))
                                   for mol in self.moles]

        num_atom_counts = atom_counts['num_atom'].value_counts()

        for count, num_mol in num_atom_counts.to_dict().items():
            if self.use_remainder:
                num_batch_for_this = -(-num_mol // self.batch_size)
            else:
                num_batch_for_this = num_mol // self.batch_size

            target_mols = atom_counts.query('num_atom==@count')['mol_index'].values
            random.shuffle(target_mols)

            devider = np.arange(0, len(target_mols), self.batch_size)
            devider = np.append(devider, 99999)

            if self.use_remainder:
                target_mols = np.append(
                    target_mols,
                    np.repeat(target_mols[-1], -len(target_mols) % self.batch_size))

            for b in range(num_batch_for_this):
                batches.append(target_mols[devider[b]:devider[b + 1]])

        random.shuffle(batches)
        batches = np.concatenate(batches).astype(np.int32)

        return batches

batch_size = 10
train_sampler = SameSizeSampler(structures_groups, train_moles, batch_size)
valid_sampler = SameSizeSampler(structures_groups, valid_moles, batch_size,
                                use_remainder=True)
test_sampler = SameSizeSampler(structures_groups, test_moles, batch_size,
                               use_remainder=True)

## Make iterators, oprimizer
Iterators for data feeding is made as below.

In [54]:
train_iter = chainer.iterators.SerialIterator(
    train_dataset, batch_size, order_sampler=train_sampler)

valid_iter = chainer.iterators.SerialIterator(
    valid_dataset, batch_size, repeat=False, order_sampler=valid_sampler)

test_iter = chainer.iterators.SerialIterator(
    test_dataset, batch_size, repeat=False, order_sampler=test_sampler)

## Make optimizer
Adam is used as an optimizer.

In [55]:
from chainer import optimizers
optimizer = optimizers.Adam(alpha=1e-3)
optimizer.setup(model)

## Make updator
Since the model receives input arrays separately, I implement an original converter.
`input_array` and `dists` are exstracted from `Graph` object and `pair_index` and `targets` are exstracted from `targets` object.
`targets` is added only for training.
When this converter is used for evaluation, `targets` is not added.

In [56]:
from chainer import training
from chainer.dataset import to_device

def coupling_converter(batch, device):

    list_array = list()
    list_dists = list()
    list_targets = list()
    list_pairs_index = list()

    with_target = 'fc' in batch[0]['targets'].columns

    for i, d in enumerate(batch):
        list_array.append(d['graphs'].input_array)
        list_dists.append(d['graphs'].dists)
        if with_target:
            list_targets.append(
                d['targets'][['fc', 'sd', 'pso', 'dso']].values.astype(np.float32))

        sample_index = np.full((len(d['targets']), 1), i)
        atom_index = d['targets'][['atom_index_0', 'atom_index_1']].values

        list_pairs_index.append(np.concatenate([sample_index, atom_index], axis=1))

    input_array = to_device(device, np.stack(list_array))
    dists = to_device(device, np.stack(list_dists))
    pairs_index = np.concatenate(list_pairs_index)

    array = {'input_array': input_array, 'dists': dists, 'pairs_index': pairs_index}

    if with_target:
        array['targets'] = to_device(device, np.concatenate(list_targets))

    return array

updater = training.StandardUpdater(train_iter, optimizer,
                                   converter=coupling_converter, device=0)
trainer = training.Trainer(updater, (10, 'epoch'), out="result")

# Training extensions
## Evaluator
I implemented an Evaluator which measure validation score during training.
The prediction for test data is also calculated in this evaluator and the submision file is generated.

In [57]:
from chainer.training.extensions import Evaluator
from chainer import cuda

class TypeWiseEvaluator(Evaluator):

    def __init__(self, iterator, target, converter, device, name,
                 is_validate=False, is_submit=False):

        super(TypeWiseEvaluator, self).__init__(
            iterator, target, converter=converter, device=device)

        self.is_validate = is_validate
        self.is_submit = is_submit
        self.name = name

    def calc_score(self, df_truth, pred):

        target_types = list(set(df_truth['type']))

        diff = df_truth['scalar_coupling_constant'] - pred

        scores = 0
        metrics = {}

        for target_type in target_types:

            target_pair = df_truth['type'] == target_type
            score_exp = np.mean(np.abs(diff[target_pair]))
            scores += np.log(score_exp)

            metrics[target_type] = scores

        metrics['ALL_LogMAE'] = scores / len(target_types)

        observation = {}
        with reporter.report_scope(observation):
            reporter.report(metrics, self._targets['main'])

        return observation

    def evaluate(self):
        iterator = self._iterators['main']
        eval_func = self._targets['main']

        iterator.reset()
        it = iterator

        y_total = []
        t_total = []

        for batch in it:
            in_arrays = self.converter(batch, self.device)
            with chainer.no_backprop_mode(), chainer.using_config('train', False):
                y = eval_func.predict(**in_arrays)

            y_data = cuda.to_cpu(y.data)
            y_total.append(y_data)
            t_total.extend([d['targets'] for d in batch])

        df_truth = pd.concat(t_total, axis=0)
        y_pred = np.sum(np.concatenate(y_total), axis=1)

        if self.is_submit:
            submit = pd.DataFrame()
            submit['id'] = df_truth['id']
            submit['scalar_coupling_constant'] = y_pred
            submit.drop_duplicates(subset='id', inplace=True)
            submit.sort_values('id', inplace=True)
            submit.to_csv('new_submission.csv', index=False)

        if self.is_validate:
            return self.calc_score(df_truth, y_pred)

        return {}

trainer.extend(
    TypeWiseEvaluator(iterator=valid_iter, target=model, converter=coupling_converter, 
                      name='valid', device=0, is_validate=True))
trainer.extend(
    TypeWiseEvaluator(iterator=test_iter, target=model, converter=coupling_converter,
                      name='test', device=0, is_submit=True))

## Other extensions
ExponentialShift is set as a learning rate scheduler.
An extension which turn off training mode is also set to deactivate normalizatoin from second epoch.

Log options are set to report the metrics.
This helps us to analyze the result of training.

In [58]:
trainer.extend(training.extensions.ExponentialShift('alpha', 0.99999))

from chainer.training import make_extension

def stop_train_mode(trigger):
    @make_extension(trigger=trigger)
    def _stop_train_mode(_):
        chainer.config.train = False
    return _stop_train_mode

trainer.extend(stop_train_mode(trigger=(1, 'epoch')))

trainer.extend(
    training.extensions.observe_value(
        'alpha', lambda tr: tr.updater.get_optimizer('main').alpha))

trainer.extend(training.extensions.LogReport())
trainer.extend(training.extensions.PrintReport(
    ['epoch', 'elapsed_time', 'main/loss', 'valid/main/ALL_LogMAE', 'alpha']))

# Training
## Run
I tuned number of epochs to prevent timeout.
SchNet tends to be underfitting, longer training makes the model better basically.

In [59]:
chainer.config.train = True
trainer.run()

epoch       elapsed_time  main/loss   valid/main/ALL_LogMAE  alpha     
1           363.132       0.764447    1.78736                0.000933569  
2           723.724       0.318716    -0.418252              0.000871542  
3           1087.55       0.207418    -0.623638              0.000813637  
4           1450.99       0.173211    -0.814682              0.000759579  
5           1814.48       0.151959    -0.804071              0.000709112  
6           2176.58       0.137095    -0.910407              0.000661998  
7           2538.83       0.125599    -1.03943               0.000618015  
8           2900.18       0.116679    -1.13283               0.000576954  
9           3267.31       0.109116    -1.2219                0.000538621  
10          3629.48       0.103322    -1.11847               0.000502835  


## Check output

In [64]:
tmp = pd.read_csv('schnet_newdata/Answer.csv')
submit = pd.read_csv('new_submission.csv')
display(submit.head())
print('shape: {}'.format(submit.shape))
display(tmp.head())
print('shape: {}'.format(tmp.shape))

,id,scalar_coupling_constant
0,198,82.890724
1,199,-2.916242
2,200,-0.079300
3,201,-10.598700
4,202,-16.576838


shape: (465203, 2)


,id,scalar_coupling_constant
0,198,82.163900
1,221,82.162000
2,220,-2.596450
3,219,-0.138109
4,218,-10.603400


shape: (465203, 2)


In [61]:
from chainer import serializers
serializers.save_npz('my.state', trainer)

In [62]:
serializers.save_npz('my.updater', updater)